# * VINSIGHT : Data Issue

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Any Issue Transaction

### Prepaid Active Caller : TMH

In [3]:
# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250401
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'TB1S000600' --Prepaid Active Caller : TMH
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    # print(f'\nCurrent Timestamp...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-04-23, 11:10:50
   -> v_start_date: 20250401

    TM_KEY_MTH  TM_KEY_DAY   METRIC_CD                              METRIC_NAME   AREA_NAME  ACTUAL_AS_OF ACTUAL_SNAP ACTUAL_AGG_MTH              PPN_TM
0       202504    20250401  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250401  17,205,721     17,205,721 2025-04-23 00:34:11
1       202504    20250402  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250402  17,200,601     17,200,601 2025-04-23 00:34:11
2       202504    20250403  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250403  17,198,038     17,198,038 2025-04-23 00:34:11
3       202504    20250404  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250404  17,166,314     17,166,314 2025-04-23 00:34:11
4       202504    20250405  TB1S000600  Prepaid Active Caller 30D Rolling : TMH  Nationwide      20250405  17,132,397     17,132,397 2025-04-23 00

### Area 080 : DTAC

In [4]:
# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME
        , SUM(CASE WHEN AREA_CD = '080' THEN ACTUAL_SNAP END) ACTUAL_080
        , SUM(CASE WHEN AREA_CD = '080' THEN TARGET_SNAP END) TARGET_080
        , SUM(CASE WHEN AREA_CD = '016' THEN ACTUAL_SNAP END) ACTUAL_016
        , SUM(CASE WHEN AREA_CD = '016' THEN TARGET_SNAP END) TARGET_016
        , SUM(CASE WHEN AREA_CD = '040' THEN ACTUAL_SNAP END) ACTUAL_040
        , SUM(CASE WHEN AREA_CD = '040' THEN TARGET_SNAP END) TARGET_040
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF, MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'DB1R000100' --Prepaid Revenue : DTAC
    --WHERE METRIC_CD = 'DB2R000100' --Postpaid Revenue : DTAC
    --WHERE METRIC_CD = 'DB2R010100' --Postpaid Revenue B2C : DTAC
    --WHERE METRIC_CD = 'DB2R020100' --Postpaid Revenue B2B : DTAC
    AND AREA_CD IN ('016', '040', '080')
    AND TM_KEY_DAY >= {v_start_date} 
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME
    ORDER BY TM_KEY_DAY, METRIC_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    mod_col_list = txn_df.iloc[:, 4:10].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=100)}')
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-04-23, 11:11:05
   -> v_start_date: 20250101

     TM_KEY_MTH  TM_KEY_DAY   METRIC_CD             METRIC_NAME ACTUAL_080 TARGET_080 ACTUAL_016 TARGET_016 ACTUAL_040 TARGET_040  ACTUAL_AS_OF              PPN_TM
0        202501    20250101  DB1R000100  Prepaid Revenue : DTAC  1,458,637        nan    362,185    365,268  1,096,452  1,060,020      20250101 2025-04-23 02:12:31
1        202501    20250102  DB1R000100  Prepaid Revenue : DTAC  1,378,370        nan    365,318    373,482  1,013,052  1,083,855      20250102 2025-04-23 02:12:31
2        202501    20250103  DB1R000100  Prepaid Revenue : DTAC  1,330,136        nan    356,411    362,361    973,725  1,051,581      20250103 2025-04-23 02:12:31
3        202501    20250104  DB1R000100  Prepaid Revenue : DTAC  1,229,670        nan    312,077    348,242    917,593  1,010,608      20250104 2025-04-23 02:12:31
4        202501    20250105  DB1R000100  Prepaid Revenue : DTAC  1,312,542        nan    32

### Prepaid Revenue : TMH

In [2]:
# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250401
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'TB1R000100' --Prepaid Revenue : TMH
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-04-25, 09:52:37
   -> v_start_date: 20250401

    TM_KEY_MTH  TM_KEY_DAY   METRIC_CD            METRIC_NAME   AREA_NAME  ACTUAL_AS_OF  ACTUAL_SNAP ACTUAL_AGG_MTH              PPN_TM
0       202504    20250401  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250401  101,725,829    101,725,829 2025-04-25 00:47:47
1       202504    20250402  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250402   92,314,263    194,040,092 2025-04-25 00:47:47
2       202504    20250403  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250403   85,596,819    279,636,911 2025-04-25 00:47:47
3       202504    20250404  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250404   85,954,445    365,591,356 2025-04-25 00:47:47
4       202504    20250405  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250405   84,406,717    449,998,074 2025-04-25 00:47:47
5       202504    20250406  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250406   80,94

## Monthly Revenue ARPU

In [2]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B7R000102' --Prepaid ARPU
        , 'DB7R000102' --Prepaid ARPU : DTAC
        , 'TB7R000102' --Prepaid ARPU : TMH
        , 'B7R000101' --Postpaid ARPU
        , 'B7R010101' --Postpaid ARPU B2C
        , 'DB7R000101' --Postpaid ARPU : DTAC
        , 'DB7R010101' --Postpaid ARPU B2C : DTAC
        , 'TB7R000101' --Postpaid ARPU : TMH
        , 'TB7R010101' --Postpaid ARPU B2C : TMH
        , 'TB7R000103' --TOL ARPU
        , 'TB7R000104' --TVS ARPU
        )
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    --ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # # Display
    # txn_df = chk_src_df.copy()
    # mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    # for col in mod_col_list:
    #     txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    # print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-04-28, 10:08:52
   -> v_start_date: 20250101


In [3]:
''' B7R000102 : Prepaid ARPU '''

v_metric_cd = 'B7R000102'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nB7R000102 : Prepaid ARPU...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B7R000102_day_df = chk_src_df.copy()
B7R000102_day_df = B7R000102_day_df.loc[B7R000102_day_df['METRIC_CD']==v_metric_cd]
# B7R000102_day_df = B7R000102_day_df.loc[B7R000102_day_df['TM_KEY_DAY']>=v_tm_key_day]
B7R000102_day_df = B7R000102_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = B7R000102_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    B7R000102_day_df[col] = B7R000102_day_df[col].apply(lambda x: format(x, ',.1f'))
    
B7R000102_day_df


B7R000102 : Prepaid ARPU...

   -> v_metric_cd: B7R000102


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,B7R000102,Prepaid ARPU,Nationwide,20250101,172.9,172.9,2025-04-27 23:40:52
1,202501,20250102,B7R000102,Prepaid ARPU,Nationwide,20250102,172.8,172.8,2025-04-27 23:40:52
2,202501,20250103,B7R000102,Prepaid ARPU,Nationwide,20250103,173.2,173.2,2025-04-27 23:40:52
3,202501,20250104,B7R000102,Prepaid ARPU,Nationwide,20250104,173.4,173.4,2025-04-27 23:40:52
4,202501,20250105,B7R000102,Prepaid ARPU,Nationwide,20250105,173.5,173.5,2025-04-27 23:40:52
...,...,...,...,...,...,...,...,...,...
111,202504,20250422,B7R000102,Prepaid ARPU,Nationwide,20250422,152.3,152.3,2025-04-27 23:40:52
112,202504,20250423,B7R000102,Prepaid ARPU,Nationwide,20250423,408.9,408.9,2025-04-27 23:40:52
113,202504,20250424,B7R000102,Prepaid ARPU,Nationwide,20250424,152.6,152.6,2025-04-27 23:40:52
114,202504,20250425,B7R000102,Prepaid ARPU,Nationwide,20250425,152.8,152.8,2025-04-27 23:40:52


In [4]:
''' B7R010101 : Postpaid ARPU B2C '''

v_metric_cd = 'B7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nB7R010101 : Postpaid ARPU B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B7R010101_day_df = chk_src_df.copy()
B7R010101_day_df = B7R010101_day_df.loc[B7R010101_day_df['METRIC_CD']==v_metric_cd]
# B7R010101_day_df = B7R010101_day_df.loc[B7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
B7R010101_day_df = B7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = B7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    B7R010101_day_df[col] = B7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
B7R010101_day_df


B7R010101 : Postpaid ARPU B2C...

   -> v_metric_cd: B7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,B7R010101,Postpaid ARPU B2C,Nationwide,20250101,416.8,416.8,2025-04-27 23:40:52
1,202501,20250102,B7R010101,Postpaid ARPU B2C,Nationwide,20250102,417.0,417.0,2025-04-27 23:40:52
2,202501,20250103,B7R010101,Postpaid ARPU B2C,Nationwide,20250103,417.0,417.0,2025-04-27 23:40:52
3,202501,20250104,B7R010101,Postpaid ARPU B2C,Nationwide,20250104,417.0,417.0,2025-04-27 23:40:52
4,202501,20250105,B7R010101,Postpaid ARPU B2C,Nationwide,20250105,416.7,416.7,2025-04-27 23:40:52
...,...,...,...,...,...,...,...,...,...
111,202504,20250422,B7R010101,Postpaid ARPU B2C,Nationwide,20250422,416.5,416.5,2025-04-27 23:40:52
112,202504,20250423,B7R010101,Postpaid ARPU B2C,Nationwide,20250423,416.6,416.6,2025-04-27 23:40:52
113,202504,20250424,B7R010101,Postpaid ARPU B2C,Nationwide,20250424,416.6,416.6,2025-04-27 23:40:52
114,202504,20250425,B7R010101,Postpaid ARPU B2C,Nationwide,20250425,393.7,393.7,2025-04-27 23:40:52


In [5]:
''' TB7R010101 : Postpaid ARPU B2C : TMH '''

v_metric_cd = 'TB7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R010101 : Postpaid ARPU B2C : TMH...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R010101_day_df = chk_src_df.copy()
TB7R010101_day_df = TB7R010101_day_df.loc[TB7R010101_day_df['METRIC_CD']==v_metric_cd]
# TB7R010101_day_df = TB7R010101_day_df.loc[TB7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R010101_day_df = TB7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R010101_day_df[col] = TB7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R010101_day_df


TB7R010101 : Postpaid ARPU B2C : TMH...

   -> v_metric_cd: TB7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250101,371.6,371.6,2025-04-27 23:40:52
1,202501,20250102,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250102,372.0,372.0,2025-04-27 23:40:52
2,202501,20250103,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250103,372.0,372.0,2025-04-27 23:40:52
3,202501,20250104,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250104,372.0,372.0,2025-04-27 23:40:52
4,202501,20250105,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250105,371.9,371.9,2025-04-27 23:40:52
...,...,...,...,...,...,...,...,...,...
111,202504,20250422,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250422,371.4,371.4,2025-04-27 23:40:52
112,202504,20250423,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250423,371.4,371.4,2025-04-27 23:40:52
113,202504,20250424,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250424,371.4,371.4,2025-04-27 23:40:52
114,202504,20250425,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250425,335.5,335.5,2025-04-27 23:40:52


In [6]:
''' DB7R010101 : Postpaid ARPU B2C : DTAC '''

v_metric_cd = 'DB7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nDB7R010101 : Postpaid ARPU B2C : DTAC...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

DB7R010101_day_df = chk_src_df.copy()
DB7R010101_day_df = DB7R010101_day_df.loc[DB7R010101_day_df['METRIC_CD']==v_metric_cd]
# DB7R010101_day_df = DB7R010101_day_df.loc[DB7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
DB7R010101_day_df = DB7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = DB7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    DB7R010101_day_df[col] = DB7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
DB7R010101_day_df


DB7R010101 : Postpaid ARPU B2C : DTAC...

   -> v_metric_cd: DB7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250101,494.7,494.7,2025-04-27 23:40:52
1,202501,20250102,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250102,494.5,494.5,2025-04-27 23:40:52
2,202501,20250103,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250103,494.6,494.6,2025-04-27 23:40:52
3,202501,20250104,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250104,494.4,494.4,2025-04-27 23:40:52
4,202501,20250105,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250105,494.1,494.1,2025-04-27 23:40:52
...,...,...,...,...,...,...,...,...,...
111,202504,20250422,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250422,496.5,496.5,2025-04-27 23:40:52
112,202504,20250423,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250423,496.6,496.6,2025-04-27 23:40:52
113,202504,20250424,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250424,496.6,496.6,2025-04-27 23:40:52
114,202504,20250425,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250425,496.6,496.6,2025-04-27 23:40:52


In [7]:
''' TB7R000103 : TOL ARPU '''

v_metric_cd = 'TB7R000103'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R000103 : TOL ARPU...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R000103_day_df = chk_src_df.copy()
TB7R000103_day_df = TB7R000103_day_df.loc[TB7R000103_day_df['METRIC_CD']==v_metric_cd]
# TB7R000103_day_df = TB7R000103_day_df.loc[TB7R000103_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R000103_day_df = TB7R000103_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R000103_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R000103_day_df[col] = TB7R000103_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R000103_day_df


TB7R000103 : TOL ARPU...

   -> v_metric_cd: TB7R000103


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R000103,TOL ARPU,Nationwide,20250101,510.9,510.9,2025-04-27 23:40:52
1,202501,20250102,TB7R000103,TOL ARPU,Nationwide,20250102,510.8,510.8,2025-04-27 23:40:52
2,202501,20250103,TB7R000103,TOL ARPU,Nationwide,20250103,510.9,510.9,2025-04-27 23:40:52
3,202501,20250104,TB7R000103,TOL ARPU,Nationwide,20250104,510.8,510.8,2025-04-27 23:40:52
4,202501,20250105,TB7R000103,TOL ARPU,Nationwide,20250105,510.6,510.6,2025-04-27 23:40:52
...,...,...,...,...,...,...,...,...,...
111,202504,20250422,TB7R000103,TOL ARPU,Nationwide,20250422,514.4,514.4,2025-04-27 23:40:52
112,202504,20250423,TB7R000103,TOL ARPU,Nationwide,20250422,0.0,514.4,2025-04-27 23:40:52
113,202504,20250424,TB7R000103,TOL ARPU,Nationwide,20250422,0.0,514.4,2025-04-27 23:40:52
114,202504,20250425,TB7R000103,TOL ARPU,Nationwide,20250422,0.0,514.4,2025-04-27 23:40:52


In [8]:
''' TB7R000104 : TVS ARPU '''

v_metric_cd = 'TB7R000104'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R000104 : Postpaid ARPU B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R000104_day_df = chk_src_df.copy()
TB7R000104_day_df = TB7R000104_day_df.loc[TB7R000104_day_df['METRIC_CD']==v_metric_cd]
# TB7R000104_day_df = TB7R000104_day_df.loc[TB7R000104_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R000104_day_df = TB7R000104_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R000104_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R000104_day_df[col] = TB7R000104_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R000104_day_df


TB7R000104 : Postpaid ARPU B2C...

   -> v_metric_cd: TB7R000104


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R000104,TVS ARPU,Nationwide,20250101,260.4,260.4,2025-04-27 23:40:52
1,202501,20250102,TB7R000104,TVS ARPU,Nationwide,20250102,259.8,259.8,2025-04-27 23:40:52
2,202501,20250103,TB7R000104,TVS ARPU,Nationwide,20250103,259.8,259.8,2025-04-27 23:40:52
3,202501,20250104,TB7R000104,TVS ARPU,Nationwide,20250104,259.9,259.9,2025-04-27 23:40:52
4,202501,20250105,TB7R000104,TVS ARPU,Nationwide,20250105,259.9,259.9,2025-04-27 23:40:52
...,...,...,...,...,...,...,...,...,...
111,202504,20250422,TB7R000104,TVS ARPU,Nationwide,20250422,258.7,258.7,2025-04-27 23:40:52
112,202504,20250423,TB7R000104,TVS ARPU,Nationwide,20250422,0.0,258.7,2025-04-27 23:40:52
113,202504,20250424,TB7R000104,TVS ARPU,Nationwide,20250422,0.0,258.7,2025-04-27 23:40:52
114,202504,20250425,TB7R000104,TVS ARPU,Nationwide,20250422,0.0,258.7,2025-04-27 23:40:52
